In [1]:
import pandas as pd
from sklearn.cluster import KMeans

In [3]:
## RFM => Regency (Último -> data - data - 1) / Frequency (quantidade -> n) / Monetary (soma/média -> R   ~x)

def rfm_variables(df):
    ## FREQUENCY
    f_sales = len(df) ## Quantidade de vendas 
    f_profit = len(df[df['Profit'] > 0]) ## Quantidade de vendas com lucros
    ## MONETARY
    m_sales = round(df['Sales'].sum(), 2) ## Total de vendas
    m_profit = round(df['Profit'].sum(), 2) ## Total de vendas com lucro
    m_quantity = df['Quantity'].sum() ## Total de quantidades
    ## PERIODICITY
    df_sort = df[['Order Date']].sort_values(by='Order Date').drop_duplicates() ## Periodicidade
    df_sort['diff'] = df_sort['Order Date'] - df_sort['Order Date'].shift(1)
    df_sort['diff_int'] = df_sort['diff'].dt.days
    ## REGENCY
    r_days = round(df_sort['diff_int'].mean(), 2)
    return f_sales, f_profit, m_sales, m_profit, m_quantity, r_days

In [4]:
def fit_data(data, variable):
    rfm = pd.DataFrame()
    variables = data[variable].unique()
    for v in variables:
        var = data[data[variable] == v]
        f_sales, f_profit, m_sales, m_profit, m_quantity, r_days = rfm_variables(var)
        rfm = rfm.append(
            {
                'reference': v,
                'm_sales': m_sales,
                'm_profit': m_profit, 
                'm_quantity': m_quantity,
                'r_days': r_days,
                'f_sales': f_sales, 
                'f_profit': f_profit
            },
            ignore_index = True
        )
    return rfm

In [5]:
data = pd.read_feather('database/base.feather')
data

,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,City,State,Region,...,OMonth,OPeriod,Order Date Month,SYear,SMonth,SPeriod,Ship Date Month,Delivery,Price,Benefit
0,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,Henderson,Kentucky,South,...,11,35,2016-11-01,2016,11,35,2016-11-01,3,130.98,1
1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,Henderson,Kentucky,South,...,11,35,2016-11-01,2016,11,35,2016-11-01,3,243.98,1
2,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,Los Angeles,California,West,...,6,30,2016-06-01,2016,6,30,2016-06-01,4,7.31,1
3,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,Fort Lauderdale,Florida,South,...,10,22,2015-10-01,2015,10,22,2015-10-01,7,191.52,0
4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,Fort Lauderdale,Florida,South,...,10,22,2015-10-01,2015,10,22,2015-10-01,7,11.18,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9989,CA-2014-110422,2014-01-21,2014-01-23,Second Class,TB-21400,Tom Boeckenhauer,Consumer,Miami,Florida,South,...,1,1,2014-01-01,2014,1,1,2014-01-01,2,8.42,1
9990,CA-2017-121258,2017-02-26,2017-03-03,Standard Class,DB-13060,Dave Brooks,Consumer,Costa Mesa,California,West,...,2,38,2017-02-01,2017,3,39,2017-03-01,5,45.98,1
9991,CA-2017-121258,2017-02-26,2017-03-03,Standard Class,DB-13060,Dave Brooks,Consumer,Costa Mesa,California,West,...,2,38,2017-02-01,2017,3,39,2017-03-01,5,129.29,1
9992,CA-2017-121258,2017-02-26,2017-03-03,Standard Class,DB-13060,Dave Brooks,Consumer,Costa Mesa,California,West,...,2,38,2017-02-01,2017,3,39,2017-03-01,5,7.40,1


In [6]:
variables = ['m_sales', 'm_profit', 'm_quantity', 'm_deliver', 'r_days', 'f_sales', 'f_profit']

state_rfm = fit_data(data, 'State')
state_rfm = state_rfm.fillna(0)
state_rfm['cluster'] = KMeans(
    n_clusters=3,
    random_state=0
).fit(
    state_rfm[
        variables
    ]
).labels_

state_rfm

/tmp/ipykernel_2068/3999741325.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rfm = rfm.append(
/tmp/ipykernel_2068/3999741325.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rfm = rfm.append(
/tmp/ipykernel_2068/3999741325.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rfm = rfm.append(
/tmp/ipykernel_2068/3999741325.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rfm = rfm.append(
/tmp/ipykernel_2068/3999741325.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rfm = rfm.append(
/tmp/ipykernel_2068/3999741325.py:7: FutureWarning

KeyError: "['m_deliver'] not in index"